Interactive Video Frame Explorer (Jupyter Notebook Version)
============================================================

This notebook provides an interactive interface for browsing and saving frames
from a video file directly within Jupyter. It uses OpenCV for frame extraction,
Matplotlib for inline display, and ipywidgets + ipyfilechooser for interactive
file and folder selection.

Features
--------
1. Click to select a video file.
2. Click to select an output folder for saving PNGs.
3. Use a slider to navigate frames interactively.
4. Save any displayed frame to the chosen folder.

If no save folder is chosen, frames are saved to the same directory as the video.

Author: Tuan Nguyen (documented and extended by ChatGPT)

Date: November 2025

In [13]:
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Button, VBox, HBox, Output
from ipyfilechooser import FileChooser
from IPython.display import display
from pathlib import Path

# --- File chooser widget ---
fc = FileChooser('.', title='📁 Select a video file')
display(fc)

out = Output()

def load_video(change):
    """
    Callback function triggered when a user selects a video file.
    
    This function:
      - Opens the selected video using OpenCV.
      - Initializes an interactive slider to browse frames.
      - Displays the current frame inline.
      - Provides a button to save the displayed frame as a PNG image.
    
    Parameters
    ----------
    change : dict
        Change event dictionary automatically passed by ipyfilechooser.
        (Unused here but required by the callback signature.)
    """
    video_path = fc.selected
    if not video_path:
        print("❌ No file selected.")
        return
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"⚠️ Could not open {video_path}")
        return
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎞️ Loaded: {Path(video_path).name} ({frame_count} frames)")

    def get_frame(idx: int):
        """
        Retrieve a specific frame from the loaded video.
        
        Parameters
        ----------
        idx : int
            Frame index to retrieve.
        
        Returns
        -------
        numpy.ndarray or None
            RGB image array of the frame, or None if the frame could not be read.
        """
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame_bgr = cap.read()
        if not ret:
            print("⚠️ Failed to read frame", idx)
            return None
        return cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    
    def show_frame(frame_idx: int):
        """
        Display a video frame inline using Matplotlib.
        
        Parameters
        ----------
        frame_idx : int
            Index of the frame to display.
        
        Returns
        -------
        numpy.ndarray or None
            The displayed frame as an RGB array.
        """
        frame_rgb = get_frame(frame_idx)
        if frame_rgb is not None:
            with out:
                out.clear_output(wait=True)
                plt.figure(figsize=(6, 4))
                plt.imshow(frame_rgb)
                plt.title(f"Frame {frame_idx}")
                plt.axis("off")
                plt.show()
        return frame_rgb
    
    # --- Slider widget ---
    frame_slider = IntSlider(
        value=0,
        min=0,
        max=frame_count - 1,
        step=1,
        description="Frame:",
        continuous_update=False,
    )
    
    # --- Save button ---
    save_btn = Button(description="💾 Save Frame", button_style="success")
    
    def on_save_clicked(b):
        """
        Save the currently displayed frame as a PNG image.
        
        Parameters
        ----------
        b : ipywidgets.Button
            The button widget that triggered the event.
        """
        idx = frame_slider.value
        frame_rgb = get_frame(idx)
        if frame_rgb is not None:
            out_path = Path(video_path).with_suffix(f".frame{idx}.png")
            plt.imsave(out_path, frame_rgb)
            print(f"✅ Saved frame {idx} → {out_path}")
    
    save_btn.on_click(on_save_clicked)
    
    # --- Display interface ---
    interact(show_frame, frame_idx=frame_slider)
    display(HBox([save_btn]))
    display(out)

# Register callback for file selection
fc.register_callback(load_video)

FileChooser(path='/projects/kumar-lab/nguyetu/OFA_analysis/notebooks', filename='', title='📁 Select a video fi…

🎞️ Loaded: 101225_Female_Cdkl5_trimmed.mp4 (108150 frames)


interactive(children=(IntSlider(value=0, continuous_update=False, description='Frame:', max=108149), Output())…

Output()